<a href="https://colab.research.google.com/github/mendesvi/COMPILADORES/blob/main/P2_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segundo Projeto: Analisador Sintático

O segundo projeto requer que você implemente um analisador sintático para a linguagem uC e gere uma árvore de sintaxe como saída (observe que a árvore de sintaxe abstrata será construída apenas no terceiro projeto). Para concluir este segundo projeto, você também usará o [SLY](https://sly.readthedocs.io/), uma versão Python do conjunto de ferramentas [lex/yacc](http://dinosaur.compilertools.net/) com a mesma funcionalidade mas com uma interface mais amigável. Por favor, leia o conteúdo completo desta seção e complete cuidadosamente as etapas indicadas.


## Visão Geral do Analisador Sintático: Construíndo uma Árvore de Sintaxe

Considere a seguinte gramática:

```
<program> ::= <statements> EOF

<statements> ::= { <statement> }+

<statement> ::= <assign_statement>
              | <if_statement>
              | <while_statement>
              | <print_statement>

<assign_statement> ::= <identifier> EQUALS <expr> SEMI

<if_statement> ::= IF <expr> LBRACE <statements> RBRACE { ELSE LBRACE <statements> RBRACE }?

<while_statement> ::= WHILE <expr> LBRACE <statements> RBRACE

<print_statement> ::= PRINT <expr> SEMI

<expr> ::= <expr> PLUS <expr>
         | <expr> TIMES <expr>
         | <expr> EQ <expr>
         | <expr> NE <expr>
         | <constant>
         | <identifier>
         | LPAREN <expr> RPAREN

<identifier> ::= ID

<constant> ::= NUM
```

Nessa gramática, a seguinte sintaxe é usada:

```
 {símbolos}*  ==> Zero ou mais repetições de símbolos
 {símbolos}+  ==> Um ou mais repetições de símbolos
 {símbolos}?  ==> Zero ou uma ocorrência de símbolos (opcional)
 sim1 | sim2  ==> Ou sim1 ou sim2 (uma escolha)
```

Alguns exemplos de sentenças válidas para essa gramática são:

```cpp
a = 3 * 4 + 5 ;
```

```cpp
print a ;
```

```cpp
a = 3;
b = 4 * a;
print (a+b);
```

```cpp
count = 0;
max = 5;
while count != max {
  count = count + 1;
  print count;
}
```

### Analisador Léxico

O primeiro passo é construir um analisador léxico para os terminais dessa gramática:

In [ ]:
!pip install sly

In [ ]:
from sly import Lexer

class MyLexer(Lexer):
    """A lexer for the Grammar."""

    # Reserved keywords
    keywords = {
        'else': "ELSE",
        'if': "IF",
        'print': "PRINT",
        'while': "WHILE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
        # Identifiers
        "ID",
        # Constants
        "NUM",
        # Operators
        "PLUS",
        "TIMES",
        "EQ",
        "NE",
        # Assignment
        "EQUALS",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACE",
        "RBRACE",  # { }
        "SEMI",    # , ;        
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Regular expression rules for tokens    
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    NUM = r'\d+'
    PLUS = r'\+'
    TIMES = r'\*'
    NE = r'!='
    EQ = r'=='
    EQUALS = r'='
    SEMI = r';'
    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACE = r'{'
    RBRACE = r'}'

    # Special cases
    def ID(self, t):
        t.type = self.keywords.get(t.value, "ID")
        return t

    def NUM(self, t):
        t.value = int(t.value)
        return t

    # Define a rule so we can track line numbers
    @_(r'\n+')
    def ignore_newline(self, t):
        self.lineno += len(t.value)

    def find_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        return token.index - last_cr

    # Error handling rule
    def error(self, t):
        print("Illegal character %s at position (%d,%d)" % \
              (repr(t.value[0]), t.lineno, self.find_column(t)))
        self.index += 1

In [ ]:
def main(args):
    if len(args) > 0:  
        lex = MyLexer()
        for tok in lex.tokenize(args[0]):
            print(tok)    

In [ ]:
main(["a = 3 * 4 + 5"])

### Reconhecendo as sentenças

Agora, vamos desenvolver um analisador sintático para reconhecer sentenças dessa gramática e construir uma árvore de sintaxe para elas. 

In [ ]:
from sly import Parser

class MyParser(Parser):
    """A parser for the Grammar."""

    # Get the token list from the lexer (required)
    tokens = MyLexer.tokens

    def __init__(self):
        self.lexer = MyLexer()

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    # <program> ::= <statements>
    @_('statements')
    def program(self, p):
        pass

    # <statements> ::= { <statement> }+
    @_('statement { statement }')
    def statements(self, p):
        pass

    # <statement> ::= <assign_statement>
    #               | <if_statement>
    #               | <while_statement>
    #               | <print_statement>
    @_('assign_statement',
       'if_statement',
       'while_statement',
       'print_statement')
    def statement(self, p):
        pass

    # <assign_statement> ::= <identifier> EQUALS <expr> SEMI
    @_('identifier EQUALS expr SEMI')
    def assign_statement(self, p):
        pass

    # <if_statement> ::= IF <expr> LBRACE <statements> RBRACE { ELSE LBRACE <statements> RBRACE }?
    @_('IF expr LBRACE statements RBRACE [ ELSE LBRACE statements RBRACE ]')
    def if_statement(self, p):
        pass

    # <while_statement> ::= WHILE <expr> LBRACE <statements> RBRACE
    @_('WHILE expr LBRACE statements RBRACE')
    def while_statement(self, p):
        pass

    # <print_statement> ::= PRINT <expr> SEMI
    @_('PRINT expr SEMI')
    def print_statement(self, p):
        pass

    # <expr> ::= <expr> PLUS <expr>
    #          | <expr> TIMES <expr>
    #          | <expr> EQ <expr>
    #          | <expr> NE <expr>
    @_('expr PLUS expr',
       'expr TIMES expr',
       'expr EQ expr',
       'expr NE expr')
    def expr(self, p):
        pass

    # <expr> ::= <constant>
    #          | <identifier>
    @_('constant',
       'identifier')
    def expr(self, p):
        pass

    # <expr> ::= LPAREN <expr> RPAREN
    @_('LPAREN expr RPAREN')
    def expr(self, p):
        pass

    # <constant> ::= NUM
    @_('NUM')
    def constant(self, p):
        pass

    # <identifier> ::= ID
    @_('ID')
    def identifier(self, p):
        pass

In [ ]:
def build_tree(root):
    return '\n'.join(_build_tree(root))

def _build_tree(node):
    if isinstance(node, list):
        if not node: return
        node = tuple(node)
    
    if not isinstance(node, tuple):
        yield " "+str(node)
        return

    values = [_build_tree(n) for n in node]
    if len(values) == 1:
        yield from build_lines('──', '  ', values[0])
        return

    start, *mid, end = values
    yield from build_lines('┬─', '│ ', start)
    for value in mid:
        yield from build_lines('├─', '│ ', value)
    yield from build_lines('└─', '  ', end)

def build_lines(first, other, values):
    try:
        yield first + next(values)
        for value in values:
            yield other + value
    except StopIteration:
        return

In [ ]:
def main(args):
    if len(args) > 0:    
        lex = MyLexer()
        parser = MyParser()
        st = parser.parse(args[0])
        if st is not None:
            print(build_tree(st))

Vamos ignorar os avisos por enquanto e tentar analisar uma sentença válida:

In [ ]:
main(["a = 3 * 4 + 5 ;"])

Nada aconteceu! Vamos ver o que acontece com uma sentença inválida:

In [ ]:
main(["a == 3 ;"])

Agora vamos adicionar informações para construir uma árvore de sintaxe:

In [ ]:
from sly import Parser

class MyParser(Parser):
    """A parser for the Grammar."""

    # Get the token list from the lexer (required)
    tokens = MyLexer.tokens

    def __init__(self):
        self.lexer = MyLexer()

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    # <program> ::= <statements>
    @_('statements')
    def program(self, p):
        return ('program', p.statements)

    # <statements> ::= { <statement> }+
    @_('statement { statement }')
    def statements(self, p):
        return [p.statement0] + p.statement1

    # <statement> ::= <assign_statement>
    #               | <if_statement>
    #               | <while_statement>
    #               | <print_statement>
    @_('assign_statement',
       'if_statement',
       'while_statement',
       'print_statement')
    def statement(self, p):
        return p[0]

    # <assign_statement> ::= <identifier> EQUALS <expr> SEMI
    @_('identifier EQUALS expr SEMI')
    def assign_statement(self, p):
        return ('assign', p.identifier, p.expr)

    # <if_statement> ::= IF <expr> LBRACE <statements> RBRACE { ELSE LBRACE <statements> RBRACE }?
    @_('IF expr LBRACE statements RBRACE [ ELSE LBRACE statements RBRACE ]')
    def if_statement(self, p):
        return ('if', p.expr, p.statements0, p.statements1)

    # <while_statement> ::= WHILE <expr> LBRACE <statements> RBRACE
    @_('WHILE expr LBRACE statements RBRACE')
    def while_statement(self, p):
        return ('while', p.expr, p.statements)

    # <print_statement> ::= PRINT <expr> SEMI
    @_('PRINT expr SEMI')
    def print_statement(self, p):
        return ('print', p.expr)

    # <expr> ::= <expr> PLUS <expr>
    #          | <expr> TIMES <expr>
    #          | <expr> EQ <expr>
    #          | <expr> NE <expr>
    @_('expr PLUS expr',
       'expr TIMES expr',
       'expr EQ expr',
       'expr NE expr')
    def expr(self, p):
        return ('expr: %s' % (p[1]), p.expr0, p.expr1)

    # <expr> ::= <constant>
    #          | <identifier>
    @_('constant',
       'identifier')
    def expr(self, p):
        return p[0]

    # <expr> ::= LPAREN <expr> RPAREN
    @_('LPAREN expr RPAREN')
    def expr(self, p):
        return p.expr

    # <constant> ::= NUM
    @_('NUM')
    def constant(self, p):
        return ('num: %s'% (p.NUM))

    # <identifier> ::= ID
    @_('ID')
    def identifier(self, p):
        return ('id: %s' % (p.ID))

In [ ]:
main(["a = 3 * 4 + 5 ;"])

Construída a árvore de sintaxe, notamos que ela não está respeitando a precedência dos operadores. Precisamos indicar isso no analisador sintático. Isso é feito adicionando a variável `precedence` à classe do analisador. Consulte a seção [“Lidando com Gramáticas Ambíguas”](https://sly.readthedocs.io/en/latest/sly.html#dealing-with-ambiguous-grammars)
da documentação do SLY. 

Vamos também aproveitar e definir uma rotina de erro para o analisador sintático:

In [ ]:
from sly import Parser

class MyParser(Parser):
    """A parser for the Grammar."""

    # Get the token list from the lexer (required)
    tokens = MyLexer.tokens

    precedence = (
        ('left', 'PLUS'),
        ('left', 'TIMES'),
        ('left', 'EQ'),
        ('left', 'NE'),
    )

    def __init__(self):
        self.lexer = MyLexer()

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    # Error handling rule
    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= <statements>
    @_('statements')
    def program(self, p):
        return ('program', p.statements)

    # <statements> ::= { <statement> }+
    @_('statement { statement }')
    def statements(self, p):
        return [p.statement0] + p.statement1

    # <statement> ::= <assign_statement>
    #               | <if_statement>
    #               | <while_statement>
    #               | <print_statement>
    @_('assign_statement',
       'if_statement',
       'while_statement',
       'print_statement')
    def statement(self, p):
        return p[0]

    # <assign_statement> ::= <identifier> EQUALS <expr> SEMI
    @_('identifier EQUALS expr SEMI')
    def assign_statement(self, p):
        return ('assign', p.identifier, p.expr)

    # <if_statement> ::= IF <expr> LBRACE <statements> RBRACE { ELSE LBRACE <statements> RBRACE }?
    @_('IF expr LBRACE statements RBRACE [ ELSE LBRACE statements RBRACE ]')
    def if_statement(self, p):
        return ('if', p.expr, p.statements0, p.statements1)

    # <while_statement> ::= WHILE <expr> LBRACE <statements> RBRACE
    @_('WHILE expr LBRACE statements RBRACE')
    def while_statement(self, p):
        return ('while', p.expr, p.statements)

    # <print_statement> ::= PRINT <expr> SEMI
    @_('PRINT expr SEMI')
    def print_statement(self, p):
        return ('print', p.expr)

    # <expr> ::= <expr> PLUS <expr>
    #          | <expr> TIMES <expr>
    #          | <expr> EQ <expr>
    #          | <expr> NE <expr>
    @_('expr PLUS expr',
       'expr TIMES expr',
       'expr EQ expr',
       'expr NE expr')
    def expr(self, p):
        return ('expr: %s' % (p[1]), p.expr0, p.expr1)

    # <expr> ::= <constant>
    #          | <identifier>
    @_('constant',
       'identifier')
    def expr(self, p):
        return p[0]

    # <expr> ::= LPAREN <expr> RPAREN
    @_('LPAREN expr RPAREN')
    def expr(self, p):
        return p.expr

    # <constant> ::= NUM
    @_('NUM')
    def constant(self, p):
        return ('num: %s'% (p.NUM))

    # <identifier> ::= ID
    @_('ID')
    def identifier(self, p):
        return ('id: %s' % (p.ID))

Com a definição de precedência de operadores, os conflitos empilha/reduz foram resolvidos. Vamos executar o analisador sintático para nossas sentenças de exemplo novamente:

In [ ]:
main(["a = 3 * 4 + 5 ;"])

In [ ]:
main(["a == 3 ; "])

Para fins de rastreamento de posição, você deve salvar o número de linha e coluna em cada produção. Consulte a seção [“Número de Linha e Rastreamento de Posição”](https://sly.readthedocs.io/en/latest/sly.html#line-number-and-position-tracking)
da documentação do SLY. Para fazer isso, sugiro extrair o número da linha e da coluna do símbolo terminal mais à esquerda de cada produção, se houver. Por exemplo:

```python
    # Internal auxiliary methods
    def _token_coord(self, p):
        return p.lineno, self.lexer.find_column(p)
```

```python
    # <assign_statement> ::= <identifier> EQUALS <expr> SEMI
    @_('identifier EQUALS expr SEMI')
    def assign_statement(self, p):
        return ('assign @ %d:%d' % self._token_coord(p), p.identifier, p.expr)
```

Vamos ajustar o analisador sintático para rastrear a posição de símbolos terminais e executá-lo em nossas sentenças de exemplo novamente:

In [ ]:
from sly import Parser

class MyParser(Parser):
    """A parser for the Grammar."""

    # Get the token list from the lexer (required)
    tokens = MyLexer.tokens

    precedence = (
        ('left', 'PLUS'),
        ('left', 'TIMES'),
        ('left', 'EQ'),
        ('left', 'NE'),
    )

    def __init__(self):
        self.lexer = MyLexer()

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    # Internal auxiliary methods
    def _token_coord(self, p):
        return p.lineno, self.lexer.find_column(p)

    # Error handling rule
    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= <statements>
    @_('statements')
    def program(self, p):
        return ('program', p.statements)

    # <statements> ::= { <statement> }+
    @_('statement { statement }')
    def statements(self, p):
        return [p.statement0] + p.statement1

    # <statement> ::= <assign_statement>
    #               | <if_statement>
    #               | <while_statement>
    #               | <print_statement>
    @_('assign_statement',
       'if_statement',
       'while_statement',
       'print_statement')
    def statement(self, p):
        return p[0]

    # <assign_statement> ::= <identifier> EQUALS <expr> SEMI
    @_('identifier EQUALS expr SEMI')
    def assign_statement(self, p):
        return ('assign @ %d:%d' % self._token_coord(p), p.identifier, p.expr)

    # <if_statement> ::= IF <expr> LBRACE <statements> RBRACE { ELSE LBRACE <statements> RBRACE }?
    @_('IF expr LBRACE statements RBRACE [ ELSE LBRACE statements RBRACE ]')
    def if_statement(self, p):
        return ('if @ %d:%d' % self._token_coord(p), p.expr, p.statements0, p.statements1)

    # <while_statement> ::= WHILE <expr> LBRACE <statements> RBRACE
    @_('WHILE expr LBRACE statements RBRACE')
    def while_statement(self, p):
        return ('while @ %d:%d' % self._token_coord(p), p.expr, p.statements)

    # <print_statement> ::= PRINT <expr> SEMI
    @_('PRINT expr SEMI')
    def print_statement(self, p):
        return ('print @ %d:%d' % self._token_coord(p), p.expr)

    # <expr> ::= <expr> PLUS <expr>
    #          | <expr> TIMES <expr>
    #          | <expr> EQ <expr>
    #          | <expr> NE <expr>
    @_('expr PLUS expr',
       'expr TIMES expr',
       'expr EQ expr',
       'expr NE expr')
    def expr(self, p):
        return ('expr: %s @ %d:%d' % ((p[1],) + self._token_coord(p)), p.expr0, p.expr1)

    # <expr> ::= <constant>
    #          | <identifier>
    @_('constant',
       'identifier')
    def expr(self, p):
        return p[0]

    # <expr> ::= LPAREN <expr> RPAREN
    @_('LPAREN expr RPAREN')
    def expr(self, p):
        return p.expr

    # <constant> ::= NUM
    @_('NUM')
    def constant(self, p):
        return ('num: %s @ %d:%d' % ((p.NUM,) + self._token_coord(p)))

    # <identifier> ::= ID
    @_('ID')
    def identifier(self, p):
        return ('id: %s @ %d:%d' % ((p.ID,) +  self._token_coord(p)))

In [ ]:
main(["a = 3 * 4 + 5 ;"])

In [ ]:
main(["a = 3 * ;"])

In [ ]:
main(["a == 3 ; "])

In [ ]:
main(["print a ;"])

In [ ]:
code = '''
a = 3;
b = 4 * a;
print (a+b);
'''

In [ ]:
main([code])

In [ ]:
code = '''
count = 0;
max = 5;
while count != max {
  count = count + 1;
  print count;
}
'''

In [ ]:
main([code])

## Escrevendo um Analisador Sintático para a Linguagem uC

Nesta etapa, você deve escrever uma versão preliminar de um analisador sintático para a linguagem uC. A especificação da gramática do uC em BNF está [aqui](https://colab.research.google.com/drive/13o4SdAWIlpDYZ6rU308LqiwXsd0dwRe5?usp=sharing). Sua tarefa é escrever regras de análise sintática usando o SLY. Para um melhor entendimento, estude o capítulo [“Escrevendo um Analisador Sintático”](https://sly.readthedocs.io/en/latest/sly.html#writing-a-parser)
da documentação do SLY.

### Especificação
Sua tarefa é traduzir as regras listadas em uma gramática BNF em uma coleção de métodos decorados pelo decorador `@_()`. O nome de cada método deve corresponder ao nome da regra gramatical que está sendo analisada. O argumento para o decorador `@_()` é uma cadeia de caracteres que descreve o lado direito da gramática. Assim, uma regra gramatical como:

```
    <program> ::= {<global_declaration>}+
```

torna-se um método de classe do Python da forma:

```python
class UCParser(Parser):
    """A parser for the uC language."""
    ...
    # <program> ::= {<global_declaration>}+
    @_('global_declaration_list')
    def program(self, p):
        return ('program', p.global_declaration_list)

    @_('global_declaration { global_declaration }')
    def global_declaration_list(self, p):
        return [p.global_declaration0] + p.global_declaration1
```

Para construir uma árvore de sintaxe, basta criar e retornar uma tupla ou lista em cada função de regra gramatical, como mostrado acima.

Seu objetivo, ao final deste segundo projeto, é reconhecer **sintaticamente** programas expressos na linguagem uC. Para isso, o ideal é que você faça com que sua gramática apresente **somente um** conflito empilha/reduz relacionado ao comando *if-else*.

**Sugestão:** Você deve começar de forma simples e trabalhar incrementalmente até construir a gramática completa.

### Esboço do Analisador Sintático

In [ ]:
import sys
!pip install sly
from sly import Lexer, Parser

Copie o código da classe `UCLexer` que você escreveu no [primeiro projeto](https://colab.research.google.com/drive/1TFZwdVJGMVDo48cq63LsWGnqC9UXx3n1?usp=sharing) e cole na célula abaixo.

In [ ]:
class UCLexer(Lexer):
    """A lexer for the uC language."""

    def __init__(self, error_func):
        """Create a new Lexer.
        An error function. Will be called with an error
        message, line and column as arguments, in case of
        an error during lexing.
        """
        self.error_func = error_func
        
    # Reserved keywords
    keywords = {
        'assert': "ASSERT",
        'break': "BREAK",
        'char': "CHAR",
        'else': "ELSE",
        'for': "FOR",
        'if': "IF",
        'int': "INT",
        'print': "PRINT",
        'read': "READ",
        'return': "RETURN",
        'void': "VOID",
        'while': "WHILE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
         # Identifiers
        "ID",
        # constants
        "INT_CONST",
        "CHAR_CONST",
        "STRING_LITERAL",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "MOD",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NE",
        "OR",
        "AND",
        "NOT",
        # Assignment
        "EQUALS",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACKET",
        "RBRACKET",  # [ ]
        "LBRACE",
        "RBRACE",  # { }
        "COMMA",
        "SEMI",  # , ;
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Other ignored patterns
    ignore_newline = r'[\\\n]'# <<< INCLUDE A REGEX HERE FOR NEWLINE >>>
    ignore_comment = r'(\/\*([\s\S]*?)\*\/)|(\/\/.*\n)'# <<< INCLUDE A REGEX HERE FOR COMMENT >>>

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][0-9a-zA-Z_]*'# <<< INCLUDE A REGEX HERE FOR ID >>>
    INT_CONST = r'[0-9]+'# <<< INCLUDE A REGEX HERE FOR INT_CONST >>>
    CHAR_CONST = r'(\'(\\.|[^\\]?)\')' # <<< INCLUDE A REGEX HERE FOR CHAR_CONST >>>
    # <<< YOUR CODE HERE >>>
    STRING_LITERAL = r'(\"(.|(\\\"))*?[^\\]\")'
    PLUS = r'\+'
    MINUS = r'\-'
    TIMES = r'\*'
    DIVIDE = r'\/(?!\*)'
    MOD = r'\%'
    LE = r'\<\='
    LT = r'\<(?!\=)'
    GE = r'\>\='
    GT = r'\>(?!\=)'
    EQ = r'\=\='
    NE = r'\!\='
    OR = r'\|\|'
    AND = r'\&\&'
    NOT = r'\!(?!\=)'
    EQUALS = r'\=(?!\=)'
        # Delimeters
    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACKET = r'\['
    RBRACKET = r'\]'
    LBRACE = r'\{'
    RBRACE = r'\}'
    COMMA = r'\,'
    SEMI = r'\;'

    #erros
    unmatched_quote = r'\"[^\"]*?(?=\n)'

    def unmatched_quote(self, t):
      msg = "Unterminated string literal"
      self._error(msg, t)

    uccomment = r'(\/\*)([\s\S]*?)[^(\*\/)]\Z'

    def uccomment(self, t):
      msg = "Unterminated comment"
      self._error(msg, t)

    Unterminated_character = r'\'[^\']*?(?=\n)|\'.{2,}\'|\'\\\''

    def Unterminated_character(self, t):
      msg = "Unterminated character const"
      self._error(msg, t)

    # Special cases
    def ID(self, t):
      t.type = self.keywords.get(t.value, "ID")
      return t

    # Define a rule so we can track line numbers
    def ignore_newline(self, t):
      self.lineno += len(t.value)
 
    def ignore_comment(self, t):
      self.lineno += t.value.count("\n")

    def find_tok_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        if last_cr < 0: last_cr = 0
        return token.index - last_cr + 1

    # Internal auxiliary methods
    def _error(self, msg, token):
        location = self._make_tok_location(token)
        self.error_func(msg, location[0], location[1])
        self.index += 1

    def _make_tok_location(self, token):
        return token.lineno, self.find_tok_column(token)

    # Error handling rule
    def error(self, t):
        msg = "Illegal character %s" % repr(t.value[0])
        self._error(msg, t)

    # Scanner (used only for test)
    def scan(self, text):
        output = ""
        for tok in self.tokenize(text):
            print(tok)
            output += str(tok) + "\n"
        return output

In [ ]:
class UCParser(Parser):
    """A parser for the uC language."""

    # Get the token list from the lexer (required)
    tokens = UCLexer.tokens

    precedence = (
        
        # <<< YOUR CODE HERE >>>
    )
    
    def __init__(self, error_func=lambda msg, x, y: print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)):
        """Create a new Parser.
        An error function for the lexer.
        """
        self.lexer = UCLexer(error_func)

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    # Internal auxiliary methods
    def _token_coord(self, p):
        return self.lexer._make_location(p)

    # Error handling rule
    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= {<global_declaration>}+
    @_('global_declaration_list')
    def program(self, p):
        return ('program', p.global_declaration_list)
        
    @_('global_declaration { global_declaration }')
    def global_declaration_list(self, p):
        return [p.global_declaration0] + p.global_declaration1        

    # <global_declaration> ::= <function_definition>
    #                        | <declaration>
    # <<< YOUR CODE HERE >>>
    @_('function_definition', 'declaration')
    def global_declaration(self, p):
        return ('global_declaration', p[0])
   
    # <function_definition> ::= <type_specifier> <declarator> {<declaration>}* <compound_statement>
    # <<< YOUR CODE HERE >>>
    @_('type_specifier declarator declaration compound_statement' )
    def function_definition(self, p):
        return ('funtion_definition', p.type_specifier, p.declarator, p.declaration, p.compound_statement )     #provavel erro
    # <type_specifier> ::= "void"
    #                    | "char"
    #                    | "int"
    # <<< YOUR CODE HERE >>>
    @_('VOID', 'CHAR', 'INT')
    def type_specifier(self, p):
        return ('type_specifier', p[0])
    # <declarator> ::= <identifier>
    #                | "(" <declarator> ")"
    #                | <declarator> "[" {<constant_expression>}? "]"
    #                | <declarator> "(" {<parameter_list>}? ")"
    # <<< YOUR CODE HERE >>>
    @_('LPAREN declarator RPAREN')
    def declarator(self, p):
        return ('declarator', "(", p.declarator, ")") #possivel erro

    @_('declarator LBRACKET [ constant_expression ] RBRACKET')
    def declarator(self, p):
        return ('declarator', "[", p.constant_expression, "]")    

    @_('declarator LPAREN [ parameter_list ] RPAREN')
    def declarator(self, p):
        return ('declarator', "(", p.parameter_list, ")")  

    @_('ID')
    def declarator(self, p):
        return ('declarator', p.identifier)  
    # <constant_expression> ::= <binary_expression>
    # <<< YOUR CODE HERE >>>
    @_('binary_expression')
    def constant_expression(self, p):
        return ('constant_expression', p.binary_expression)

    # <binary_expression> ::= <unary_expression>
    #                       | <binary_expression>  "*"   <binary_expression>
    #                       | <binary_expression>  "/"   <binary_expression>
    #                       | <binary_expression>  "%"   <binary_expression>
    #                       | <binary_expression>  "+"   <binary_expression>
    #                       | <binary_expression>  "-"   <binary_expression>
    #                       | <binary_expression>  "<"   <binary_expression>
    #                       | <binary_expression>  "<="  <binary_expression>
    #                       | <binary_expression>  ">"   <binary_expression>
    #                       | <binary_expression>  ">="  <binary_expression>
    #                       | <binary_expression>  "=="  <binary_expression>
    #                       | <binary_expression>  "!="  <binary_expression>
    #                       | <binary_expression>  "&&"  <binary_expression>
    #                       | <binary_expression>  "||"  <binary_expression>
    # <<< YOUR CODE HERE >>>
    @_('unary_expression')
    def binary_expression(self, p):
        return ('binary_expression', p.unary_expression)

    @_('binary_expression TIMES binary_expression', 
       'binary_expression DIVIDE binary_expression', 
       'binary_expression MOD binary_expression',
       'binary_expression PLUS binary_expression',
       'binary_expression MINUS binary_expression', 
       'binary_expression LT binary_expression',
       'binary_expression LE binary_expression',
       'binary_expression GT binary_expression',
       'binary_expression GE binary_expression',
       'binary_expression EQ binary_expression',
       'binary_expression NE binary_expression',
       'binary_expression AND binary_expression',
       'binary_expression OR binary_expression')
    def binary_expression(self, p):
        return ('binary_expression', p.binary_expression, p[1], p.binary_expression)   

    # <unary_expression> ::= <postfix_expression>
    #                      | <unary_operator> <unary_expression>
    # <<< YOUR CODE HERE >>>
    @_('postfix_expression')
    def unary_expression(self, p):
        return ('unary_expression', p.postfix_expression)

    @_('unary_operator unary_expression')
    def unary_expression(self, p):
        return ('unary_expression', p.unary_operator, p.unary_expression)   

    # <postfix_expression> ::= <primary_expression>
    #                        | <postfix_expression> "[" <expression> "]"
    #                        | <postfix_expression> "(" {<argument_expression>}? ")"
    # <<< YOUR CODE HERE >>>
    @_('primary_expression')
    def postfix_expression(self, p):
        return ('postfix_expression', p.primary_expression)

    @_('postfix_expression "[" expression "]"')
    def postfix_expression(self, p):
        return ('postfix_expression', p.postfix_expression, "[", p.expression, "]")   

    @_('postfix_expression LPAREN [ argument_expression ] RPAREN ')
    def postfix_expression(self, p):
        return ('postfix_expression', p.postfix_expression, "(", p.argument_expression, ")")  

    # <primary_expression> ::= <identifier>
    #                        | <constant>
    #                        | <string>
    #                        | "(" <expression> ")"
    # <<< YOUR CODE HERE >>>
    @_('ID', 'constant', 'STRING_LITERAL')
    def primary_expression(self, p):
        return ('primary_expression', p[0])

    @_('LPAREN expression RPAREN')
    def primary_expression(self, p):
        return ('primary_expression', "(", p.expression, ")")

    # <constant> ::= <integer_constant>
    #              | <character_constant>
    # <<< YOUR CODE HERE >>>
    @_('INT_CONST', 'CHAR_CONST')
    def constant(self, p):
        return ('constant', p.integer_constant, p.character_constant)

    # <expression> ::= <assignment_expression>
    #                | <expression> "," <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def expression(self, p):
        return ('expression', p.assignment_expression)
    @_('expression COMMA assignment_expression')
    def expression(self, p):
        return ('expression', p.expression, COMMA, p.assignment_expression)    

    # <argument_expression> ::= <assignment_expression>
    #                         | <argument_expression> "," <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def argument_expression(self, p):
        return ('argument_expression', p.assignment_expression)
    @_('argument_expression COMMA assignment_expression')
    def argument_expression(self, p):
        return ('argument_expression', p.argument_expression, COMMA, p.assignment_expression)  

    # <assignment_expression> ::= <binary_expression>
    #                           | <unary_expression> "=" <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('binary_expression')
    def assignment_expression(self, p):
        return ('assignment_expression', p.binary_expression)

    @_('unary_expression EQUALS assignment_expression')
    def assignment_expression(self, p):
        return ('assignment_expression', p.unary_expression, EQUALS, p.assignment_expression)
    # <unary_operator> ::= "+"
    #                    | "-"
    #                    | "!"
    # <<< YOUR CODE HERE >>>
    @_('PLUS', 'MINUS', 'NOT')
    def unary_operator(self, p):
        return ('unary_operator', PLUS, MINUS, NOT) 

    # <parameter_list> ::= <parameter_declaration>
    #                    | <parameter_list> "," <parameter_declaration>
    # <<< YOUR CODE HERE >>>
    @_('parameter_declaration')
    def parameter_list(self, p):
        return ('parameter_list', p.parameter_declaration)

    @_('parameter_list COMMA parameter_declaration')
    def parameter_list(self, p):
        return ('parameter_list', p.parameters_list, p.parameter_declaration)   

    # <parameter_declaration> ::= <type_specifier> <declarator>
    # <<< YOUR CODE HERE >>>
    @_('type_specifier declarator')
    def parameter_declaration(self, p):
        return ('parameter_declaration', p.type_specifier, p.declarator)

    # <declaration> ::=  <type_specifier> {<init_declarator_list>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('type_specifier [ init_declarator_list ] SEMI')
    def declaration(self, p):
        return ('declaration', p.type_specifier, p.init_declarator_list, SEMI)

    # <init_declarator_list> ::= <init_declarator>
    #                          | <init_declarator_list> "," <init_declarator>
    # <<< YOUR CODE HERE >>>
    @_('init_declarator')
    def init_declarator_list(self, p):
        return ('init_declarator_list', p.init_declarator)
    @_('init_declarator_list COMMA init_declarator')
    def init_declarator_list(self, p):
        return ('init_declarator_list', p.init_declarator_list, COMMA, p.init_declarator)  

    # <init_declarator> ::= <declarator>
    #                     | <declarator> "=" <initializer>
    # <<< YOUR CODE HERE >>>
    @_('declarator')
    def init_declarator(self, p):
        return ('init_declarator', p.declarator)

    @_('declarator EQUALS initializer')
    def init_declarator(self, p):
        return ('init_declarator', p.init_declarator, EQUALS, p.initializer)

    # <initializer> ::= <assignment_expression>
    #                 | "{" {<initializer_list>}? "}"
    #                 | "{" <initializer_list> , "}"
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def initializer(self, p):
        return (p.assignment_expression)
    @_('LPAREN [ initializer_list ] RPAREN')
    def initializer(self, p):
        return (p.initializer_list)   
    @_('LPAREN initializer_list COMMA RPAREN')
    def initializer(self, p):
        return (p.initializer_list)     

    # <initializer_list> ::= <initializer>
    #                      | <initializer_list> "," <initializer>
    # <<< YOUR CODE HERE >>>
    @_('initializer')
    def initializer_list(self, p):
        return (p.initializer_list) 

    @_('initializer_list COMMA initializer')
    def initializer_list(self, p):
        return (p.initializer_list, p.initializer)     

    # <compound_statement> ::= "{" {<declaration>}* {<statement>}* "}"
    # <<< YOUR CODE HERE >>>
    @_('LBRACE { declaration } { statement } RBRACE')
    def compound_statement(self, p):
        return ('compound_statement', p.declaration, p.statement) 
    # <statement> ::= <expression_statement>
    #               | <compound_statement>
    #               | <selection_statement>
    #               | <iteration_statement>
    #               | <jump_statement>
    #               | <assert_statement>
    #               | <print_statement>
    #               | <read_statement>
    # <<< YOUR CODE HERE >>>
    @_('expression_statement', 'compound_statement', 'selection_statement', 'iteration_statement', 'jump_statement', 'assert_statement', 'print_statement', 'read_statement')
    def statement(self, p):
        return (p[0]) 
    # <expression_statement> ::= {<expression>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('[ expression ] SEMI')
    def expression_statement(self, p):
        return (p.expression) 

    # <selection_statement> ::= "if" "(" <expression> ")" <statement>
    #                         | "if" "(" <expression> ")" <statement> "else" <statement>
    # <<< YOUR CODE HERE >>>
    @_('IF LPAREN expression RPAREN statement')
    def selection_statement(self, p):
        return ('if @ %d:%d' % self._token_coord(p), p.expression, p.statement) 
    @_('IF LPAREN expression RPAREN statement ELSE statement')
    def selection_statement(self, p):
        return ('if @ %d:%d' % self._token_coord(p), p.expression, p.statement0, p.statement1) 

    # <iteration_statement> ::= "while" "(" <expression> ")" <statement>
    #                         | "for" "(" {<expression>}? ";" {<expression>}? ";" {<expression>}? ")" <statement>
    #                         | "for" "(" <declaration> {<expression>}? ";" {<expression>}? ")" <statement>
    # <<< YOUR CODE HERE >>>
    @_('WHILE LPAREN expression RPAREN statement')
    def iteration_statement(self, p):
        return ('while @ %d:%d' % self._token_coord(p), p.expression, p.statement)

    @_('FOR LPAREN [ expression ] SEMI [ expression ] SEMI [ expression ] RPAREN statement')
    def iteration_statement(self, p):
        return ('for @ %d:%d' % self._token_coord(p), p.expression0, p.expression1, p.expression2, p.statement)

    @_('FOR LPAREN declaration [ expression ] SEMI [ expression ] RPAREN statement')
    def iteration_statement(self, p):
        return ('for @ %d:%d' % self._token_coord(p), p.declaration, p.expression0, p.expression1, p.statement)    

    # <jump_statement> ::= "break" ";"
    #                    | "return" {<expression>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('BREAK SEMI')
    def jump_statement(self, p):
        return ('break @ %d%d' % self._token_coord(p))

    @_('RETURN [ expression ] SEMI')
    def jump_statement(self, p):
        return ('return @ %d%d' % self._token_coord(p))

    # <assert_statement> ::= "assert" <expression> ";"
    # <<< YOUR CODE HERE >>>
    @_('ASSERT expression SEMI')
    def assert_statement(self, p):
        return ('assert @ %d:%d' % self._token_coord(p), p.expression)

    # <print_statement> ::= "print" "(" {<expression>}? ")" ";"
    # <<< YOUR CODE HERE >>>
    @_('PRINT LPAREN [ expression ] RPAREN SEMI')
    def print_statement(self, p):
        return ('print @ %d:%d' % self._token_coord(p), p.expression)

    # <read_statement> ::= "read" "(" <argument_expression> ")" ";"
    # <<< YOUR CODE HERE >>>
    @_('READ LPAREN argument_expression RPAREN SEMI')
    def read_statement(self, p):
        return ('read @ %d:%d' % self._token_coord(p), p.argument_expression)

In [ ]:
def build_tree(root):
    return '\n'.join(_build_tree(root))

def _build_tree(node):
    if isinstance(node, list):
        if not node: return
        node = tuple(node)
    
    if not isinstance(node, tuple):
        yield " "+str(node)
        return

    values = [_build_tree(n) for n in node]
    if len(values) == 1:
        yield from build_lines('──', '  ', values[0])
        return

    start, *mid, end = values
    yield from build_lines('┬─', '│ ', start)
    for value in mid:
        yield from build_lines('├─', '│ ', value)
    yield from build_lines('└─', '  ', end)

def build_lines(first, other, values):
    try:
        yield first + next(values)
        for value in values:
            yield other + value
    except StopIteration:
        return

def print_error(msg, x, y):
    # use stdout to match with the output in the .out test files
    print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)
    
def main(args):
    parser = UCParser(print_error)
    with open(args[0], 'r') if len(args) > 0 else sys.stdin as f:
        st = parser.parse(f.read())
        if st is not None:
            print(build_tree(st))

## Teste
Para o desenvolvimento inicial, tente executar o analisador sintático em um arquivo de entrada de exemplo, como:

```cpp
/* comment */
int j = 3;
int main () {
  int i = j;
  int k = 3;
  int p = 2 * j;
  assert p == 2 * i;
}
```

In [ ]:
%%file test.uc 
/* comment */
int j = 3;
int main () {
  int i = j;
  int k = 3;
  int p = 2 * j;
  assert p == 2 * i;
}

E o resultado será semelhante ao texto mostrado abaixo.

```
┬─ program
└─┬─┬─ global_declaration
  │ └─┬─ declaration
  │   ├─ type_specifier: int @ 2:1
  │   └───┬─ init_declarator
  │       ├─ identifier: j @ 2:5
  │       └─ constant: int, 3 @ 2:9
  └─┬─ function_definition
    ├─ type_specifier: int @ 3:1
    ├─┬─ function_declarator
    │ ├─ identifier: main @ 3:5
    │ └─ None
    └─┬─ compound_statement @ 3:13
      ├─┬─┬─ declaration
      │ │ ├─ type_specifier: int @ 4:3
      │ │ └───┬─ init_declarator
      │ │     ├─ identifier: i @ 4:7
      │ │     └─ identifier: j @ 4:11
      │ ├─┬─ declaration
      │ │ ├─ type_specifier: int @ 5:3
      │ │ └───┬─ init_declarator
      │ │     ├─ identifier: k @ 5:7
      │ │     └─ constant: int, 3 @ 5:11
      │ └─┬─ declaration
      │   ├─ type_specifier: int @ 6:3
      │   └───┬─ init_declarator
      │       ├─ identifier: p @ 6:7
      │       └─┬─ binary_expression: * @ 6:13
      │         ├─ constant: int, 2 @ 6:11
      │         └─ identifier: j @ 6:15
      └───┬─ assert @ 7:3
          └───┬─ binary_expression: == @ 7:12
              ├─ identifier: p @ 7:10
              └─┬─ binary_expression: * @ 7:17
                ├─ constant: int, 2 @ 7:15
                └─ identifier: i @ 7:19
```

In [ ]:
main(["test.uc"])

Estude cuidadosamente a saída do analisador sintático e certifique-se de que ela faz sentido. Quando estiver razoavelmente satisfeito com a saída, tente executar alguns dos testes mais complicados projetados para testar vários cenários atípicos, fora do padrão esperado. Você pode usar como base os exemplos contidos [aqui](https://colab.research.google.com/drive/13o4SdAWIlpDYZ6rU308LqiwXsd0dwRe5?usp=sharing). 

No [AVA](https://ava2.ead.ufscar.br/mod/quiz/view.php?id=523692) há um grande conjunto de testes para verificar seu código: confira-os para ver mais exemplos.

## Envie seu trabalho
Depois de concluir esta tarefa, copie o código da classe `UCParser` e o submeta no [AVA](https://ava2.ead.ufscar.br/mod/quiz/view.php?id=523692).

## Anexo
A lista abaixo define os nós da árvore de sintaxe que devem ser retornados em cada regra da gramática:

```
program = tuple('program', list(global_declaration))

global_declaration = function_definition 
                   | tuple('global_declaration', declaration)

function_definition = tuple('function_definition', type_specifier, declarator, declaration, compound_statement)

type_specifier = tuple('type_specifier: void @ lineno:column')
               | tuple('type_specifier: char @ lineno:column')
               | tuple('type_specifier: int @ lineno:column')

declarator = tuple('identifier: ' + str(ID) + ' @ lineno:column')
           | declarator
           | tuple('array_declarator', declarator, constant_expression)
           | tuple('function_declarator', declarator, parameter_list)

constant_expression = binary_expression

binary_expression = unary_expression
                  | tuple('binary_expression: * @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: / @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: % @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: + @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: - @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: < @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: <= @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: > @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: >= @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: == @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: != @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: && @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_expression: || @ lineno:column', binary_expression0, binary_expression1)

unary_expression = postfix_expression
                 | tuple('unary_expression: ' + str(unary_operator), unary_expression)

postfix_expression = primary_expression
                   | tuple('array_reference', postfix_expression, expression)
                   | tuple('function_call', postfix_expression, argument_expression)

primary_expression = tuple('identifier: ' + str(ID) + ' @ lineno:column')
                   | constant
                   | tuple('constant: string, ' + str(STRING_LITERAL) + ' @ lineno:column')
                   | expression

constant = integer_constant = tuple('constant: int, ' + str(INT_CONST) + ' @ lineno:column')
         | tuple('constant: char, ' + str(CHAR_CONST) + ' @ lineno:column')

expression = list(assignment_expression)
               
argument_expression = list(assignment_expression)

assignment_expression = binary_expression
                      | tuple('assign @ lineno:column', unary_expression, assignment_expression)

unary_operator = tuple('+ @ lineno:column')
               | tuple('- @ lineno:column')
               | tuple('! @ lineno:column')

parameter_list = list(parameter_declaration)

parameter_declaration = tuple('parameter_declaration', type_specifier, declarator)

declaration = tuple('declaration', type_specifier, init_declarator_list)

init_declarator_list = list(init_declarator)

init_declarator = tuple('init_declarator', declarator, initializer)

initializer = assignment_expression
            | initializer_list
            | initializer_list

initializer_list = list(initializer)

compound_statement = tuple('compound_statement @ lineno:column', declaration, statement)

statement = expression_statement
          | compound_statement
          | selection_statement
          | iteration_statement
          | jump_statement
          | assert_statement
          | print_statement
          | read_statement

expression_statement = expression

selection_statement = tuple('if @ lineno:column', expression, statement0, statement1)

iteration_statement = tuple('while @ lineno:column', expression, statement)
                    | tuple('for @ lineno:column', expression0, expression1, expression2, statement)
                    | tuple('for @ lineno:column', declaration, expression0, expression1, statement)

jump_statement = tuple('break @ lineno:column')
               | tuple('return @ lineno:column', expression)

assert_statement = tuple('assert @ lineno:column', expression)

print_statement = tuple('print @ lineno:column', expression)

read_statement = tuple('read @ lineno:column', argument_expression)
```

Um novo nó é criado sempre que o valor retornado for uma tupla do Python, por exemplo:

```python
    # <program> ::= {<global_declaration>}+
    @_('global_declaration_list')
    def program(self, p):
        return ('program', p.global_declaration_list)
```

Uma lista de nós é criada sempre que o valor retornado for uma lista do Python, por exemplo:

```python
    @_('global_declaration { global_declaration }')
    def global_declaration_list(self, p):
        return [p.global_declaration0] + p.global_declaration1  
```

Uma referência para um nó é criada sempre que o valor retornado em uma regra for o nome de outra regra.

Os valores indicados como `lineno` e `column` referem-se aos números de linha e coluna, respectivamente, do símbolo terminal mais à esquerda de cada produção, se houver.